# TODO

- lav similarity matrix på lsh pakken som er i deepnote 
- repliker lsh pakkens resultater i løsningsscript med tf-idf 
- lav recomendation funktion 
- wordclouds 
- lav notebooken lækker med beskrivelser af metoder


# load data med mrjob

In [176]:
%load_ext autoreload
%autoreload 2
!python -m pip install -r requirements.txt
from utils.initialization import *


# med mrjob
# names, party and twitter id
from Data.twitter_ids import twitter_ids
data = pd.DataFrame(columns=['name', "party", 'twitter_id'])
i = 0
for party in twitter_ids:
    for person in twitter_ids[party]:
        data.loc[i, :] = [person, party, twitter_ids[party][person]]
        i += 1

# tweets
filename = "Data/cleaned_data.csv"
if not os.path.exists(filename):
    os.system(f"python utils/clean_data_mrjob.py Data/tweets > Data/tmp_cleaned_data.txt")
    data_ = pd.DataFrame(columns=["name", "tweets"])
    with open("Data/tmp_cleaned_data.txt", "rb") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            line = eval(line.decode())
            data_.loc[i,"name"] = list(line.keys())[0]
            data_.loc[i, "tweets"] = list(line.values())[0]
    data_.to_csv(filename, index = False)

data_ = pd.read_csv(filename)
data = data.merge(data_)
data.tweets = [eval(t) for t in data.tweets]
data["tokens"] = [[w for w in word_tokenize(" ".join(data["tweets"][i])) if w.isalnum()] for i in range(len(data))]
import simplemma #use simplemma instead of nltk.WordNetLemmatizer()          ### vi har addet denne linje
data["tokens"] = [[simplemma.lemmatize(w, lang='da') for w in data["tokens"][i]] for i in range(len(data))] ### vi har addet denne linje

# only include unique words
data['tokens'] = data['tokens'].apply(lambda x: list(set(x)))

data


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


You should consider upgrading via the 'c:\Users\bayka\Anaconda3\envs\katrine_personal_env\python.exe -m pip install --upgrade pip' command.


,name,party,twitter_id,tweets,tokens
0,magnus_heunicke,socialdemokratiet,22695562,[afsætter året styrke hjælpen børn pårørende a...,"[british, landsplan, baltisk, vist, missed, kv..."
1,nicolai_wammen,socialdemokratiet,2803948786,[dage siden sagde nyt ejendomsvurderingssystem...,"[bent, mn, baltisk, vist, træ, splitte, elysee..."
2,mattias_tesfaye,socialdemokratiet,546254893,[this is literally the same logic many th c am...,"[bent, snitte, vist, maradona, udlændingenævne..."
3,jakob_ellemann,venstre,155584627,[tide få fleksibel genåbning vores børn ældre ...,"[stén, vrede, ministertid, vist, anne, varig, ..."
4,soren_gade,venstre,975064362359623680,[kære marianne synes burde læse lovforslaget i...,"[tusindvis, mirakelkur, antisemitisme, betting..."
5,sophie_lohde,venstre,44611200,[flertallet veto dermed røde partier stort set...,"[skræmmeka, des, vist, anmeldelse, underernæri..."
6,lars_lokke,moderaterne,26201346,[mon ikke sjov form argumentation mangler lidt...,"[udgiftslov, bent, des, vist, folketingsspørgs..."
7,jacob_mark,sf,2373406198,[slår fast syvtommersøm kom så godt igennem fo...,"[landsplan, snitte, vist, olieboring, dragsted..."
8,pia_dyhr,sf,65025162,"[stemmer nok selvom synes gør godt klaus, brug...","[baltisk, vist, stefan, træ, hand, anne, samir..."
9,kirsten_andersen,sf,235646319,[arbejde få medarbejdere ser virkeligheden sun...,"[bent, landsplan, julegudstjenester, vist, træ..."


In [177]:
mini_df = data #data.sample(5, random_state=42)
# add party to the name
mini_df['name'] = mini_df['name'] + ' ' + mini_df['party']

# Starter på at implementere solutions 

## Shingles

In [178]:
def shingle(aString, q, delimiter=' '):
    """
    Input:
        - aString (str): string to split into shingles
        - q (int)
        - delimiter (str): string of the delimiter to consider to split the input string (default: space)
    Return: list of unique shingles
    """
    all_shingles = []
    if delimiter != '':
        words_list = aString.split(delimiter)
    else:
        words_list = aString
    for i in range (len(words_list)-q+1):
        all_shingles.append(delimiter.join(words_list[i:i+q]))
    return list(set(all_shingles))

    
# Example from the Book
# ex_string, q = test, 3
# ex_shingles = shingle(ex_string, q, delimiter='')
# print('Initial string:', ex_string)
# print(f'>> Shingles with q = {q} :',ex_shingles)

# Example from the HINT
for i in range(len(mini_df)):
    ex_string, q = ' '.join(mini_df['tokens'][i]), 2
    ex_shingles = shingle(ex_string, q)
    # assert len(ex_shingles) == 7
    # add shingle to the dataframe
    mini_df.loc[i, 'shingles'] = str(ex_shingles)

    # print('\nInitial string:', ex_string)
    # print(f'>> Shingles with q = {q} :',ex_shingles)

# ex_string, q = 'test', 2
# ex_shingles = shingle(ex_string, q)
# assert len(ex_shingles) == 7
# print('\nInitial string:', ex_string)
# print(f'>> Shingles with q = {q} :',ex_shingles)

Now each shingle is a list of `q` words. We can use the `hash` function to convert each shingle into a number. We can then use the `min` function to find the smallest hash value for each document. This is the signature for the document.

# Lav TF-IDF til en kolonne i dataframen

In [179]:
# According to the book (9.2.2) we can get the n most important (highest tfidf score) words PER document and treat them as a set
# (unlike cosine similarity that treats the vectors with 0s and 1s for each time a word appears in both documents)
#https://www.analyticsvidhya.com/blog/2021/12/how-to-extract-key-phrases-using-tfidf-with-python/

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=700, stop_words='english')
vectors = vectorizer.fit_transform(mini_df.tokens.apply(lambda x: " ".join(x)))

In [180]:
vectors

<33x700 sparse matrix of type '<class 'numpy.float64'>'
	with 21305 stored elements in Compressed Sparse Row format>

In [181]:
dict_of_tokens={i[1]:i[0] for i in vectorizer.vocabulary_.items()}

In [182]:
tfidf_vectors = []  # all deoc vectors by tfidf
for row in vectors:
  tfidf_vectors.append({dict_of_tokens[column]:value for (column,value) in zip(row.indices,row.data)})
print("number of documents",len(tfidf_vectors), "the dictionary of document 1", len(tfidf_vectors[3].keys()))

number of documents 33 the dictionary of document 1 677


In [183]:
mini_df['TFIDF_Words'] = [list(tfidf_vectors[i].keys()) for i in range(len(mini_df))]
# mini_df['TFIDF_Words'].iloc[0]

# Minhashing algorithm
Now we implement the minhashing algorithm. `minhash` that takes a list of shingles and a seed for the hash function
mapping the shingles, and outputs the minhash

In [184]:
import sys
import os
import mmh3

#hashes a list of strings
def listhash(l,seed):
	val = 0
	for e in l:
		val = val ^ mmh3.hash(e, seed)
	return val 


#### TODO her tager vi istedet for shingles_list lister af tf-idfs

In [185]:
def minhash(shingles_list, seed):
    """
    Input:
        - shingles_list (list of str): set of hashes
        - seed (int): seed for listhash function
    Return: minhash of given shingles
    """
    minhash_value = None
    for aShingle in shingles_list:
        hashcode = listhash([aShingle], seed)
        if minhash_value == None or hashcode < minhash_value:
            minhash_value = hashcode
    return minhash_value

In [186]:
# print(f'MinHash of mini_df.shingles[0]:', minhash(mini_df.shingles[4], 42))

In [187]:
def minhash2(shingles_list, k):
    """
    Input:
        - shingles_list (list of str): set of hashes
        - k (int): seed for listhash function
    Return: sequence of k minhashes
    """
    all_minhash = []
    for i in range(k):
        all_minhash.append(minhash(shingles_list, i))
    return all_minhash

In [188]:
# k=42
# print(f'MinHash of  with k = {k}:\n', minhash2(mini_df.shingles[3], k))

## Do this for all and save the minhash value to the dataframe 



In [189]:
# for i in range(len(mini_df)):
#     mini_df.loc[i, 'minhash_tokens'] = minhash(mini_df.tokens[i], 10)

# for i in range(len(mini_df)):
#     mini_df.loc[i, 'minhas_tf'] = minhash(mini_df.TFIDF_Words[i], 10)

# Signatures

In [190]:
k=10
# def signature(dict_docs, q = q, num_hashes = k):
def signature(dict_docs, num_hashes = k):

    """
    Input:
        - dict_docs (dict of str:str): dictionary of {title:document}
        - q (int)
        - num_hashes (int)
    Return: dictionary consisting of document id’s as keys and signatures as values
    """
    dict_signatures = {}
    total_texts = len(list(dict_docs.keys()))
    counter = 1
    for key,text in dict_docs.items():
        print(f'{counter}/{total_texts} - {key} - Processing...')
        # doc_shingles = shingle(text, q)
        doc_shingles = mini_df.tokens[counter-1]
        # doc_shingles = mini_df.TFIDF_Words[counter-1]
        minhash_values = minhash2(doc_shingles, num_hashes)
        dict_signatures[key] = minhash_values
        counter += 1
    return dict_signatures

In [220]:
# dict_docs is a dictionary of {name:TFIDF_Words}
# dict_docs = {i:j for i,j in zip(mini_df['name'],mini_df['TFIDF_Words'])}
# dict_docs = {i:j for i,j in zip(mini_df['name'],mini_df['tokens'])}
dict_docs = {i:j for i,j in zip(mini_df['party'],mini_df['tokens'])}



# signature_dict = signature(dict_docs, q = 2, num_hashes = 100)
signature_dict = signature(dict_docs, num_hashes = 200)

1/12 - socialdemokratiet - Processing...
2/12 - venstre - Processing...
3/12 - moderaterne - Processing...
4/12 - sf - Processing...
5/12 - danmarksdemokraterne - Processing...
6/12 - liberal_alliance - Processing...
7/12 - konservative - Processing...
8/12 - enhedslisten - Processing...
9/12 - radikale - Processing...
10/12 - nye_borgerlige - Processing...
11/12 - alternativet - Processing...
12/12 - dansk_folkeparti - Processing...


# Jaccard similarity

In [ ]:
# def jaccard_similarity(x,y):
#   """ returns the jaccard similarity between two lists """
#   intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
#   union_cardinality = len(set.union(*[set(x), set(y)]))
#   return intersection_cardinality/float(union_cardinality)

In [223]:
def jaccard(name1, name2, signature_dict):
    """
    Input:
        - name1 (str): key of the first document S
        - name2 (str): key of the second document T
        - signatures_dict (dict of str:list): dictionary of signatures
    Return: Jaccard similarity between S and T
    """
    signatures_doc1 = np.array(signature_dict[name1])
    signatures_doc2 = np.array(signature_dict[name2])
    # return np.sum(signatures_doc1 == signatures_doc2) #/ len(signatures_doc1)
    return len(np.intersect1d(signatures_doc1, signatures_doc2))/len(np.union1d(signatures_doc1, signatures_doc2))#, np.setdiff1d(signatures_doc1, signatures_doc2)

In [224]:
# signature_dict['soren_pape']
# signature_dict

In [226]:

first_doc_key = mini_df.party[1]#list(docs.keys())[0]
second_doc_key = mini_df.party[2]#list(docs.keys())[1]
print(f'Jaccard similarity between {first_doc_key} and {second_doc_key}:', jaccard(first_doc_key, second_doc_key, signature_dict))#dict_signatures_docs))

Jaccard similarity between socialdemokratiet and socialdemokratiet: 1.0


# similarity

In [227]:
def similar(signatures_dict, jaccard_threshold=0.05):
    """
    Input:
        - signatures_dict (dict of str:list): dictionary of signatures
        - jaccard_threshold (float): lower bound for Jaccard similarity score to consider
            two documents as similar
    Return: dictionary of similar items
    """
    list_keys = list(signatures_dict.keys())
    similar_items = {}
    for i in range (len(list_keys)-1):
        for j in range (i+1, len(list_keys)):
            similarity_score = jaccard(list_keys[i], list_keys[j], signatures_dict)
            if similarity_score >= jaccard_threshold:
                similar_items[(list_keys[i], list_keys[j])] = similarity_score
    return similar_items

In [228]:
found_similar_items = similar(signature_dict)
# print('Found similar items:\n', found_similar_items)

In [229]:
# get the most similar items
most_similar_items = sorted(found_similar_items.items(), key=lambda x: x[1], reverse=True)
# print('Most similar items:\n', most_similar_items[0:20])

In [230]:
# for every person, return the 3 people most similar to him/her
def most_similar_persons(similar_items, num_similar_persons=3):
    """
    Input:
        - similar_items (dict of tuple:str): dictionary of similar items
        - num_similar_persons (int): number of similar persons to return
    Return: dictionary of most similar persons
    """
    most_similar_persons = {}
    for key,value in similar_items.items():
        if key[0] not in most_similar_persons:
            most_similar_persons[key[0]] = [(key[1], value)]
        else:
            most_similar_persons[key[0]].append((key[1], value))
        if key[1] not in most_similar_persons:
            most_similar_persons[key[1]] = [(key[0], value)]
        else:
            most_similar_persons[key[1]].append((key[0], value))
    for key,value in most_similar_persons.items():
        most_similar_persons[key] = sorted(value, key=lambda x: x[1], reverse=True)[:num_similar_persons]
    return most_similar_persons

most_similar_persons_res = most_similar_persons(found_similar_items)

In [231]:
# convert most_similar_persons_res to a datamframe for easier visualization
most_similar_persons_df = pd.DataFrame.from_dict(most_similar_persons_res, orient='index')
most_similar_persons_df.columns = ['Most similar person 1', 'Most similar person 2', 'Most similar person 3']
most_similar_persons_df

,Most similar person 1,Most similar person 2,Most similar person 3
socialdemokratiet,"(nye_borgerlige, 0.1396011396011396)","(sf, 0.12359550561797752)","(enhedslisten, 0.12359550561797752)"
venstre,"(alternativet, 0.14613180515759314)","(sf, 0.1267605633802817)","(radikale, 0.11731843575418995)"
moderaterne,"(dansk_folkeparti, 0.13314447592067988)","(sf, 0.12994350282485875)","(liberal_alliance, 0.1267605633802817)"
sf,"(liberal_alliance, 0.19402985074626866)","(konservative, 0.17994100294985252)","(radikale, 0.17647058823529413)"
liberal_alliance,"(sf, 0.19402985074626866)","(radikale, 0.1695906432748538)","(nye_borgerlige, 0.16279069767441862)"
konservative,"(sf, 0.17994100294985252)","(dansk_folkeparti, 0.15606936416184972)","(nye_borgerlige, 0.14942528735632185)"
enhedslisten,"(nye_borgerlige, 0.17647058823529413)","(sf, 0.15942028985507245)","(liberal_alliance, 0.14942528735632185)"
radikale,"(sf, 0.17647058823529413)","(nye_borgerlige, 0.17647058823529413)","(liberal_alliance, 0.1695906432748538)"
nye_borgerlige,"(enhedslisten, 0.17647058823529413)","(radikale, 0.17647058823529413)","(sf, 0.16279069767441862)"
alternativet,"(sf, 0.15273775216138327)","(venstre, 0.14613180515759314)","(liberal_alliance, 0.1396011396011396)"


In [215]:

# make a group of names for each party (for visualization)
def make_party_groups(df, party_column_name):
    """
    Input:
        - df (pd.DataFrame): dataframe with the party column
        - party_column_name (str): name of the party column
    Return: dictionary of party groups
    """
    party_groups = {}
    for party in df[party_column_name].unique():
        party_groups[party] = list(df[df[party_column_name] == party].name)
    return party_groups

party_groups = make_party_groups(mini_df, 'party')

In [217]:
# # look at the party of the most similar items
# similar_items_names = [i for i in found_similar_items.keys()]
# for i in range(len(similar_items_names)):
#     print(mini_df[mini_df.name.isin(similar_items_names[i])].party)

# LSH

In [219]:
b,r = 20, 5
# assert k == b*r

def lsh(signatures_dict, jaccard_threshold=0.05, seed=10):
    lsh_dict = {}
    for key, values in signatures_dict.items():
        blocks = np.split(np.array(values), b)
        blocks_hash_values = []
        for aBlock in blocks:
            blocks_hash_values.append(mmh3.hash(aBlock, seed))
        lsh_dict[key] = blocks_hash_values
    list_keys = list(lsh_dict.keys())
    similar_items = {}
    for i in range (len(list_keys)-1):
        for j in range (i+1, len(list_keys)):
            common_values = np.intersect1d(lsh_dict[list_keys[i]], lsh_dict[list_keys[j]])
            if len(common_values) > 0:
                # we found a candidate
                similarity_score = jaccard(list_keys[i], list_keys[j], signatures_dict)
                if similarity_score >= jaccard_threshold:
                    similar_items[(list_keys[i], list_keys[j])] = similarity_score
    return similar_items
found_similar_items_with_lsh = lsh(signature_dict)
print('Found similar items with LSH:\n', found_similar_items_with_lsh)

Found similar items with LSH:
 {}


In [205]:
# converting a string of text into a vector. Using teh transformer BERT model

# Step one: Use BERT to convert our text into a vector
# Step two:Get the cosine similarity (the cosine of the angle between the two vectors) 
    # of a fixed twitter profiles (vector) and all the other ones
# Step three: Pick the twitter profiles (vectors) with the largest cosine similarity.




In [ ]:
# https://towardsdatascience.com/hands-on-content-based-recommender-system-using-python-1d643bf314e4

def give_recommendations(index,print_recommendation = False,print_recommendation_plots= False,print_parties =False):
  index_recomm =cos_sim_data.loc[index].sort_values(ascending=False).index.tolist()[1:10]
  party_recomm =  giant_df['Party'].loc[index_recomm].values
  result = {'PArty':party_recomm,'Index':index_recomm}
  if print_recommendation==True:
    print('The watched movie is this one: %s \n'%(giant_df['Person'].loc[index]))
    k=1
    for movie in party_recomm:
      print('The number %i recommended movie is this one: %s \n'%(k,movie))
  if print_recommendation_plots==True:
    print('The plot of the watched movie is this one:\n %s \n'%(giant_df['CT'].loc[index]))
    k=1
    for q in range(len(party_recomm)):
      plot_q = giant_df['Overview'].loc[index_recomm[q]]
      print('The plot of the number %i recommended movie is this one:\n %s \n'%(k,plot_q))
      k=k+1
  if print_parties==True:
    print('The party of the twitter profile is this one:\n %s \n'%(giant_df['Party'].loc[index]))
    k=1
    for q in range(len(party_recomm)):
      plot_q = giant_df['Party'].loc[index_recomm[q]]
      print('The plot of the number %i recommended party is this one:\n %s \n'%(k,plot_q))
      k=k+1
  return result



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=30797f9c-952e-45b4-98d4-31c9ac73ae78' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>